# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# initial imports
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN


# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77536 entries, 0 to 77535
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         77536 non-null  float64
 1   interest_rate     77536 non-null  float64
 2   homeowner         77536 non-null  object 
 3   borrower_income   77536 non-null  int64  
 4   debt_to_income    77536 non-null  float64
 5   num_of_accounts   77536 non-null  int64  
 6   derogatory_marks  77536 non-null  int64  
 7   total_debt        77536 non-null  int64  
 8   loan_status       77536 non-null  object 
dtypes: float64(3), int64(4), object(2)
memory usage: 5.3+ MB


# Split the Data into Training and Testing

In [5]:
# Create our features
X = pd.get_dummies(df.drop('loan_status',1)) # convert homeowner column to category columns

# Create our target
y = df['loan_status'].values.reshape(-1,1)

In [6]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values
y[:5]

array([['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ['low_risk']], dtype=object)

In [8]:
# Create X_train, X_test, y_train, y_test
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1)
classifier = LogisticRegression(solver='lbfgs', random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# Simple Logistic Regression

In [12]:
# Create a Logistic Regression model
slr_model = classifier.fit(X_train, y_train)

In [13]:
# Calculated the balanced accuracy score
y_slr_pred = slr_model.predict(X_test)
slr_bac = balanced_accuracy_score(y_test, y_slr_pred)
print('Simple Logistic Regression Model BalancedAccuracy Score',slr_bac)

Simple Logistic Regression Model BalancedAccuracy Score 0.9889115309798473


In [14]:
# Display the confusion matrix
log_reg_cm = pd.DataFrame(confusion_matrix(y_test, y_slr_pred),index=['Actual 0','Actual 1'], columns=['Predicted 0','Predicted 1'])
print('Logistic Regression Confusion Matrix')
display(log_reg_cm)

Logistic Regression Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,609,10
Actual 1,113,18652


In [15]:
# Print the imbalanced classification report
slr_report = classification_report_imbalanced(y_test, y_slr_pred)
print(slr_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.91      0.99      0.98       619
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the following steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler()
X_ros_resampled, y_ros_resampled = ros.fit_resample(X_train,y_train)

# View the count of target classes with Counter
Counter(y_ros_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [17]:
# Train the Logistic Regression model using the resampled data
ros_model = classifier.fit(X_ros_resampled, y_ros_resampled)

In [18]:
# Calculated the balanced accuracy score
y_ros_pred = ros_model.predict(X_test)
ros_bac = balanced_accuracy_score(y_test,y_ros_pred)
print('Naive Random Oversampling Model Balanced Accuracy Score',ros_bac)

Naive Random Oversampling Model Balanced Accuracy Score 0.9934383134311076


In [19]:
# Display the confusion matrix
ros_cm = pd.DataFrame(confusion_matrix(y_test,y_ros_pred),index=['Actual 0','Actual 1'], columns=['Predicted 0','Predicted 1'])
print('Random OverSampling Confusion Matrix')
display(ros_cm)

Random OverSampling Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,125,18640


In [20]:
# Print the imbalanced classification report
ros_report = classification_report_imbalanced(y_test,y_ros_pred)
print(ros_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
smote = SMOTE(random_state=1,sampling_strategy=1.0)
X_smote_resampled,y_smote_resampled= smote.fit_resample(X_train,y_train)
# View the count of target classes with Counter
Counter(y_smote_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [22]:
# Train the Logistic Regression model using the resampled data
smote_model = classifier.fit(X_smote_resampled, y_smote_resampled)

In [23]:
# Calculated the balanced accuracy score
y_smote_pred = smote_model.predict(X_test)
smote_bac = balanced_accuracy_score(y_test,y_smote_pred)
print('SMOTE Oversampling Model Balanced Accuracy Score',smote_bac)

SMOTE Oversampling Model Balanced Accuracy Score 0.9934649587814939


In [24]:
# Display the confusion matrix
smote_cm = pd.DataFrame(confusion_matrix(y_test,y_smote_pred),index=['Actual 0','Actual 1'], columns=['Predicted 0','Predicted 1'])
print('SMOTE Model Confusion Matrix')
display(smote_cm)

SMOTE Model Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,124,18641


In [25]:
# Print the imbalanced classification report
smote_report = classification_report_imbalanced(y_test,y_smote_pred)
print(smote_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [26]:
# Resample the data using the ClusterCentroids resampler
X_cc_resampled,y_cc_resampled=ClusterCentroids(random_state=1).fit_resample(X_train,y_train)
# View the count of target classes with Counter
Counter(y_cc_resampled)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [27]:
# Train the Logistic Regression model using the resampled data
cc_model = classifier.fit(X_cc_resampled,y_cc_resampled)

In [28]:
# Calculate the balanced accuracy score
y_cc_pred = cc_model.predict(X_test)
cc_bac = balanced_accuracy_score(y_test,y_cc_pred)
print('Cluster Centroids (Undersampling) Model Balanced Accuracy Score',cc_bac)


Cluster Centroids (Undersampling) Model Balanced Accuracy Score 0.9929503031930944


In [29]:
# Display the confusion matrix
cc_cm = pd.DataFrame(confusion_matrix(y_test,y_cc_pred),index=['Actual 0','Actual 1'], columns=['Predicted 0','Predicted 1'])
print('Undersampling (Cluster Centroids) Model Confusion Matrix')
display(cc_cm)

Undersampling (Cluster Centroids) Model Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,614,5
Actual 1,113,18652


In [30]:
# Print the imbalanced classification report
cc_report = classification_report_imbalanced(y_test,y_cc_pred)
print(cc_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the training data with SMOTEENN

X_comb_resampled,y_comb_resampled=SMOTEENN(random_state=1).fit_resample(X_train,y_train)
# View the count of target classes with Counter
Counter(y_comb_resampled)

Counter({'high_risk': 55596, 'low_risk': 55934})

In [32]:
# Train the Logistic Regression model using the resampled data
comb_lr_model = classifier.fit(X_comb_resampled,y_comb_resampled)
comb_lr_model

LogisticRegression(random_state=1)

In [33]:
# Calculate the balanced accuracy score
y_comb_pred = comb_lr_model.predict(X_test)
comb_bac = balanced_accuracy_score(y_test,y_comb_pred)
print('SMOTEENN Model Balanced Accuracy Score',comb_bac)

SMOTEENN Model Balanced Accuracy Score 0.9934649587814939


In [34]:
# Display the confusion matrix
comb_cm = pd.DataFrame(confusion_matrix(y_test,y_comb_pred),index=['Actual 0','Actual 1'], columns=['Predicted 0','Predicted 1'])
print('SMOTEENN Model Confusion Matrix')
display(comb_cm)

SMOTEENN Model Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,124,18641


In [35]:
# Print the imbalanced classification report
comb_report = classification_report_imbalanced(y_test,y_comb_pred)
print(comb_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



In [36]:
print("Simple Logistic Regression:       ", slr_bac)
print("Cluster Centroid (Undersampling): ", cc_bac)
print("Random Oversampling:              ", ros_bac)
print("SMOTE Oversampling:               ", smote_bac)
print("Combined Over and Under Sampling: ", comb_bac)



Simple Logistic Regression:        0.9889115309798473
Cluster Centroid (Undersampling):  0.9929503031930944
Random Oversampling:               0.9934383134311076
SMOTE Oversampling:                0.9934649587814939
Combined Over and Under Sampling:  0.9934649587814939


In [37]:
print("Simple Logistic Regression: ")
print(slr_report)
print("SMOTE: ")
print(smote_report)
print("Random Oversampling: ")
print(ros_report)
print("Undersampling: ")
print(cc_report)
print("Combined (Over / Under) Sampling:")
print(comb_report)


Simple Logistic Regression: 
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.91      0.99      0.98       619
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384

SMOTE: 
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384

Random Oversampling: 
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.

# Final Questions

1. Which model had the best balanced accuracy score?

* *The Undersampling and Combined (Over and Under) Sampling models produced the best balance accuracy scores (0.9934649587814939).*

2. Which model had the best recall score?

* *Each of the resampling models (Combined, Random Oversampling, SMOTE algorithm, and Undersampling), each generated the highest recall for both high_risk and low_risk (99%).*

3. Which model had the best geometric mean score?

* *All the models generated a 99% geometric mean score.*
